# Check / Install Dependencies

In [1]:
# It is best to get all of these packages installed in your conda environment before running this notebook.

# Python dependencies
import os, sys
import subprocess
import numpy
import parmed
import six
import torch
import bs4
import mechanize
import psutil
import ase
import matplotlib.pyplot as plt

# InterMol installation
cwd = os.getcwd()
os.chdir(os.path.join('../2_structure_generation/Convert_Gromacs_LAMMPS','InterMol'))
os.system('python setup.py build')
os.system('python setup.py install')
os.chdir(cwd)

# GROMACS dependency
gmx_exec = subprocess.check_output('which gmx', shell=True).decode()[:-1]
assert gmx_exec.endswith('gromacs/bin/gmx')

# wget dependency
wget_exec = subprocess.check_output('which wget', shell=True).decode()[:-1]
assert wget_exec.endswith('bin/wget')

# unzip dependency
unzip_exec = subprocess.check_output('which unzip', shell=True).decode()[:-1]
assert unzip_exec.endswith('bin/unzip')

# Flag
dependencies = True

print('Required dependencies detected')

running build
running build_py
copying intermol/version.py -> build/lib/intermol
running egg_info
writing intermol.egg-info/PKG-INFO
writing dependency_links to intermol.egg-info/dependency_links.txt
writing entry points to intermol.egg-info/entry_points.txt
writing requirements to intermol.egg-info/requires.txt
writing top-level names to intermol.egg-info/top_level.txt
reading manifest file 'intermol.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
adding license file 'LICENSE.md'
writing manifest file 'intermol.egg-info/SOURCES.txt'
running install


/root/miniconda3/envs/torchkick/lib/python3.7/site-packages/setuptools/dist.py:151: UserWarning: newlines not allowed and will break in the future
  warnings.warn("newlines not allowed and will break in the future")


running bdist_egg
running egg_info
writing intermol.egg-info/PKG-INFO
writing dependency_links to intermol.egg-info/dependency_links.txt
writing entry points to intermol.egg-info/entry_points.txt
writing requirements to intermol.egg-info/requires.txt
writing top-level names to intermol.egg-info/top_level.txt
reading manifest file 'intermol.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
adding license file 'LICENSE.md'
writing manifest file 'intermol.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/intermol
creating build/bdist.linux-x86_64/egg/intermol/tests
copying build/lib/intermol/tests/README.md -> build/bdist.linux-x86_64/egg/intermol/tests
copying build/lib/intermol/tests/testing_tools.py -> build/bdist.linux-x86_64/egg/intermol/tests
creating build/bdist.linux-x86_64/egg/intermol/tests/lammps
creating build/bdist.linux-x86_

/root/miniconda3/envs/torchkick/lib/python3.7/site-packages/setuptools/dist.py:151: UserWarning: newlines not allowed and will break in the future
  warnings.warn("newlines not allowed and will break in the future")


# Ligand Force Field Parametrization & 3T Structure Generation

In [2]:
assert (dependencies==True)
from exec_workflow import main as main_exec

config_folder = 'Config_Files/Paper_Aggressive_Configs/'

def makedir(dirname):
    if not os.path.isdir(dirname):
        os.mkdir(dirname)
    return

def cleanup_folder(path1):
    dirs = os.listdir(path1)
    for dir_ in dirs:
        path2 = os.path.join(path1,dir_)
        if os.path.isdir(path2):
            dirs2 = os.listdir(path2)
            for dir2_ in dirs2:
                full_path = os.path.join(path2,dir2_)
                if os.path.isdir(full_path):
                    os.system('rm -r '+full_path)

makedir('Converted_Structures')
makedir('Minimized_Structures')

## Uncomment the block below if you want to clean up the 'Converted_Structures' and 'Minimized_Structures' folders.
## We recommend cleaning up if you are starting a fresh run, and commenting these out if your session crashes due
## to external problems and would like to continue from where you left off.

cleanup_folder('Converted_Structures')
cleanup_folder('Minimized_Structures')

print('Temporary folders have been cleaned up (or left alone if the block above is uncommented)')

Temporary folders have been cleaned up (or left alone if the block above is uncommented)


## Bad Initial CDK2 Pose Example

In [3]:
# In this example, we take 1 ligand from the CDK2 dataset with bad initial pose (RMSD>10A)
# Step 1: We give this ligand big initial rotation (no center shift) and relax it for 10k steps. Protein is frozen.
# Step 2: We give the protein-ligand pocket a gentle kick. Then minimize pocket for 5k steps.
# Step 3: Some pockets are not fully relaxed yet by this point. Minimize pocket for additional 5k steps, no kick.

main_exec(config_folder+'CDK2_aggressive_3steps.json')
os.listdir('Minimized_Structures/1fin_ENS_dH/bad_CDK2/cocry_3r9h_Z67_LIG_dock_1FIN_1/')
print('Aggressive 3T on CDK2 with bad ligand initial pose complete')

Utilizing Config_Files/Paper_Aggressive_Configs/CDK2_aggressive_3steps.json config file...
Config file content:
{'protein': '1fin_ENS_dH', 'complex_epochs': [10000, 5000, 5000], 'init_shift': [0, 0, 0], 'kick_center': [-13.094694, 205.03195, 114.42654], 'kick_radius': 25.0, 'kick_strength': 5.0, 'print_freq': 10, 'kick_seeds': [86498]}

We are using 1 GPU
Working on protein:
1fin_ENS_dH
Pocket docked with ligand:
bad_CDK2/cocry_3r9h_Z67_LIG_dock_1FIN_1
Parametrizing ligand force field and converting to GROMACS-LAMMPS format...


rm: cannot remove 'charmm36-feb2021.ff': Is a directory
                     :-) GROMACS - gmx pdb2gmx, 2021.3 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pasca

Select the Force Field:

From current directory:

 1: CHARMM36 all-atom force field (July 2020)

From '/usr/local/gromacs/share/gromacs/top':

 2: AMBER03 protein, nucleic AMBER94 (Duan et al., J. Comp. Chem. 24, 1999-2012, 2003)

 3: AMBER94 force field (Cornell et al., JACS 117, 5179-5197, 1995)

 4: AMBER96 protein, nucleic AMBER94 (Kollman et al., Acc. Chem. Res. 29, 461-469, 1996)

 5: AMBER99 protein, nucleic AMBER94 (Wang et al., J. Comp. Chem. 21, 1049-1074, 2000)

 6: AMBER99SB protein, nucleic AMBER94 (Hornak et al., Proteins 65, 712-725, 2006)

 7: AMBER99SB-ILDN protein, nucleic AMBER94 (Lindorff-Larsen et al., Proteins 78, 1950-58, 2010)

 8: AMBERGS force field (Garcia & Sanbonmatsu, PNAS 99, 2782-2787, 2002)

 9: CHARMM27 all-atom force field (CHARM22 plus CMAP for proteins)

10: GROMOS96 43a1 force field

11: GROMOS96 43a2 force field (improved alkane dihedrals)

12: GROMOS96 45a3 force field (Schuler JCC 2001 22 1205)

13: GROMOS96 53a5 force field (JCC 2004 vol 25 pag

replace LIG.mol2? [y]es, [n]o, [A]ll, [N]one, [r]ename:                      :-) GROMACS - gmx editconf, 2021.3 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pasc

In py file
/home/jpmailoa/Paper_Aggressive_3T/2_structure_generation/Convert_Gromacs_LAMMPS/InterMol/intermol/convert.py


INFO 2023-03-10 07:25:31 Beginning InterMol conversion

INFO 2023-03-10 07:25:31 InterMol is research software. If you make use of InterMol in scientific publications please cite the following reference:
Shirts, M.R., Klein, C., Swails, J.M. et al. J Comput Aided Mol Des (2016). doi:10.1007/s10822-016-9977-1

INFO 2023-03-10 07:26:39 Finished!
rm: cannot remove 'LIG_ff': Is a directory
rm: cannot remove 'charmm36-feb2021.ff': Is a directory


3T energetic kick & pocket relaxation...
Energetic kick random seed: 86498
Step:0
Step:10
Step:20
Step:30
Step:40
Step:50
Step:60
Step:70
Step:80
Step:90
Step:100
Step:110
Step:120
Step:130
Step:140
Step:150
Step:160
Step:170
Step:180
Step:190
Step:200
Step:210
Step:220
Step:230
Step:240
Step:250
Step:260
Step:270
Step:280
Step:290
Step:300
Step:310
Step:320
Step:330
Step:340
Step:350
Step:360
Step:370
Step:380
Step:390
Step:400
Step:410
Step:420
Step:430
Step:440
Step:450
Step:460
Step:470
Step:480
Step:490
Step:500
Step:510
Step:520
Step:530
Step:540
Step:550
Step:560
Step:570
Step:580
Step:590
Step:600
Step:610
Step:620
Step:630
Step:640
Step:650
Step:660
Step:670
Step:680
Step:690
Step:700
Step:710
Step:720
Step:730
Step:740
Step:750
Step:760
Step:770
Step:780
Step:790
Step:800
Step:810
Step:820
Step:830
Step:840
Step:850
Step:860
Step:870
Step:880
Step:890
Step:900
Step:910
Step:920
Step:930
Step:940
Step:950
Step:960
Step:970
Step:980
Step:990
Step:1000
Step:1010
Step:1020
Step:1

## Exploration of Alternative ADRB2 Vestibular Binding Modes from Initial Orthosteric Site

In [4]:
# In this example, we take 1 smina pose self-docked onto beta2 GPCR receptor of ADRB2 protein co-crystal (3nya). 
# Step 1: We give this ligand big initial rotation (no center shift) + large translation shift onto vestibular site,
# and relax it for 10k steps. Protein is frozen.
# Step 2: We give the protein-ligand pocket a gentle kick. Then minimize pocket for 5k steps.
# Step 3: Some pockets are not fully relaxed yet by this point. Minimize pocket for additional 5k steps, no kick.

main_exec(config_folder+'ADRB2_aggressive_3steps.json')
os.listdir('Minimized_Structures/3nya/nearby_pocket/3nya_JTZ_dock_1/')
print('Aggressive 3T on ADRB2-alprenolol with kick into extracellular vestibular binding site complete')

Utilizing Config_Files/Paper_Aggressive_Configs/ADRB2_aggressive_3steps.json config file...
Config file content:
{'protein': '3nya', 'complex_epochs': [10000, 5000, 5000], 'init_shift': [-3.0, -9.0, -9.0], 'kick_center': [-2.203958342472712, 3.352791659456367, -51.65841658910116], 'kick_radius': 25.0, 'kick_strength': 5.0, 'print_freq': 10, 'kick_seeds': [13001]}

We are using 1 GPU
Working on protein:
3nya
Pocket docked with ligand:
nearby_pocket/3nya_JTZ_dock_1
Parametrizing ligand force field and converting to GROMACS-LAMMPS format...


rm: cannot remove 'charmm36-feb2021.ff': Is a directory
                     :-) GROMACS - gmx pdb2gmx, 2021.3 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pasca

Select the Force Field:

From current directory:

 1: CHARMM36 all-atom force field (July 2020)

From '/usr/local/gromacs/share/gromacs/top':

 2: AMBER03 protein, nucleic AMBER94 (Duan et al., J. Comp. Chem. 24, 1999-2012, 2003)

 3: AMBER94 force field (Cornell et al., JACS 117, 5179-5197, 1995)

 4: AMBER96 protein, nucleic AMBER94 (Kollman et al., Acc. Chem. Res. 29, 461-469, 1996)

 5: AMBER99 protein, nucleic AMBER94 (Wang et al., J. Comp. Chem. 21, 1049-1074, 2000)

 6: AMBER99SB protein, nucleic AMBER94 (Hornak et al., Proteins 65, 712-725, 2006)

 7: AMBER99SB-ILDN protein, nucleic AMBER94 (Lindorff-Larsen et al., Proteins 78, 1950-58, 2010)

 8: AMBERGS force field (Garcia & Sanbonmatsu, PNAS 99, 2782-2787, 2002)

 9: CHARMM27 all-atom force field (CHARM22 plus CMAP for proteins)

10: GROMOS96 43a1 force field

11: GROMOS96 43a2 force field (improved alkane dihedrals)

12: GROMOS96 45a3 force field (Schuler JCC 2001 22 1205)

13: GROMOS96 53a5 force field (JCC 2004 vol 25 pag

replace LIG.mol2? [y]es, [n]o, [A]ll, [N]one, [r]ename:                      :-) GROMACS - gmx editconf, 2021.3 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pasc

In py file
/home/jpmailoa/Paper_Aggressive_3T/2_structure_generation/Convert_Gromacs_LAMMPS/InterMol/intermol/convert.py


INFO 2023-03-10 09:01:19 Beginning InterMol conversion

INFO 2023-03-10 09:01:19 InterMol is research software. If you make use of InterMol in scientific publications please cite the following reference:
Shirts, M.R., Klein, C., Swails, J.M. et al. J Comput Aided Mol Des (2016). doi:10.1007/s10822-016-9977-1

INFO 2023-03-10 09:02:19 Finished!
rm: cannot remove 'LIG_ff': Is a directory
rm: cannot remove 'charmm36-feb2021.ff': Is a directory


3T energetic kick & pocket relaxation...
Energetic kick random seed: 13001
Step:0
Step:10
Step:20
Step:30
Step:40
Step:50
Step:60
Step:70
Step:80
Step:90
Step:100
Step:110
Step:120
Step:130
Step:140
Step:150
Step:160
Step:170
Step:180
Step:190
Step:200
Step:210
Step:220
Step:230
Step:240
Step:250
Step:260
Step:270
Step:280
Step:290
Step:300
Step:310
Step:320
Step:330
Step:340
Step:350
Step:360
Step:370
Step:380
Step:390
Step:400
Step:410
Step:420
Step:430
Step:440
Step:450
Step:460
Step:470
Step:480
Step:490
Step:500
Step:510
Step:520
Step:530
Step:540
Step:550
Step:560
Step:570
Step:580
Step:590
Step:600
Step:610
Step:620
Step:630
Step:640
Step:650
Step:660
Step:670
Step:680
Step:690
Step:700
Step:710
Step:720
Step:730
Step:740
Step:750
Step:760
Step:770
Step:780
Step:790
Step:800
Step:810
Step:820
Step:830
Step:840
Step:850
Step:860
Step:870
Step:880
Step:890
Step:900
Step:910
Step:920
Step:930
Step:940
Step:950
Step:960
Step:970
Step:980
Step:990
Step:1000
Step:1010
Step:1020
Step:1